## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import random
import statistics

# генерируем массив A с N=1млн случайных целых чисел на отрезке от 0 до 1000
N = 1000000
A = [random.randint(0, 1000) for _ in range(N)]

# создаем массив B
B = [num + 100 for num in A]

# вычисляем среднее значение массива B
mean_B = statistics.mean(B)

print(mean_B)

599.864634


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import pandas as pd
import numpy as np
import string

# создаем таблицу с 2млн строк и 4 столбцами со случайными числами
N = 2000000
df = pd.DataFrame(np.random.randint(0, 1000, size=(N, 4)), columns=list('ABCD'))

# добавляем столбец key с случайными выборками из первых пяти букв английского алфавита
df['key'] = np.random.choice(list(string.ascii_uppercase)[:5], N)

print(df.head())

     A    B    C    D key
0  323  712   62  886   B
1  888  328   34   97   E
2  503  481   59  170   C
3  920   90   88  158   C
4  716  167  332  161   C


## Лабораторная работа 3

In [8]:
import pandas as pd


In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [14]:
recipes = pd.read_csv('recipes_sample.csv').dropna()
reviews = pd.read_csv('reviews_sample.csv').dropna()

In [15]:
print(recipes.dtypes)
print('')
print(reviews.dtypes)

name               object
id                  int64
minutes             int64
contributor_id      int64
submitted          object
n_steps           float64
description        object
n_ingredients     float64
dtype: object

Unnamed: 0     int64
user_id        int64
recipe_id      int64
date          object
rating         int64
review        object
dtype: object


In [16]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
reviews['date'] = pd.to_datetime(reviews['date'])

In [17]:
import pandas as pd
import time


def calculate_mean_rating_a(reviews):
    count = 0
    ratings_sum = 0
    for _, row in reviews.iterrows():
        if pd.to_datetime(row['date']).year == 2010:
            ratings_sum += row['rating']
            count += 1
    return ratings_sum / count


def calculate_mean_rating_b(reviews):
    count = 0
    ratings_sum = 0
    for _, row in reviews[reviews['date'].dt.year == 2010].iterrows():
        ratings_sum += row['rating']
        count += 1
    return ratings_sum / count


def calculate_mean_rating_c(reviews):
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()


recipes = pd.read_csv('recipes_sample.csv', index_col=0)
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

reviews['rating'] = reviews['rating'].astype(float)
reviews['date'] = pd.to_datetime(reviews['date'])


start_time = time.time()
mean_rating_a = calculate_mean_rating_a(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод A): {mean_rating_a} Время выполнения: {end_time-start_time}")

start_time = time.time()
mean_rating_b = calculate_mean_rating_b(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод B): {mean_rating_b} Время выполнения: {end_time-start_time}")

start_time = time.time()
mean_rating_c = calculate_mean_rating_c(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод C): {mean_rating_c} Время выполнения: {end_time-start_time}")

Средний рейтинг (метод A): 4.4544402182900615 Время выполнения: 6.9655444622039795
Средний рейтинг (метод B): 4.4544402182900615 Время выполнения: 0.5262711048126221
Средний рейтинг (метод C): 4.4544402182900615 Время выполнения: 0.023606061935424805


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [4]:
!pip install  line_profiler
%load_ext line_profiler
!pip install memory_profiler
%load_ext memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [18]:
%lprun -f calculate_mean_rating_b calculate_mean_rating_b(reviews)

Timer unit: 1e-09 s

Total time: 0.981097 s
File: <ipython-input-17-72b63062a01d>
Function: calculate_mean_rating_b at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    15                                           def calculate_mean_rating_b(reviews):
    16         1       5566.0   5566.0      0.0      count = 0
    17         1        255.0    255.0      0.0      ratings_sum = 0
    18     12094  852368609.0  70478.6     86.9      for _, row in reviews[reviews['date'].dt.year == 2010].iterrows():
    19     12094  123713745.0  10229.3     12.6          ratings_sum += row['rating']
    20     12094    5006373.0    414.0      0.5          count += 1
    21         1       2139.0   2139.0      0.0      return ratings_sum / count

In [19]:
%lprun -f calculate_mean_rating_a calculate_mean_rating_b(reviews)

Timer unit: 1e-09 s

Total time: 0 s
File: <ipython-input-17-72b63062a01d>
Function: calculate_mean_rating_a at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     5                                           def calculate_mean_rating_a(reviews):
     6                                               count = 0
     7                                               ratings_sum = 0
     8                                               for _, row in reviews.iterrows():
     9                                                   if pd.to_datetime(row['date']).year == 2010:
    10                                                       ratings_sum += row['rating']
    11                                                       count += 1
    12                                               return ratings_sum / count

In [20]:
%lprun -f calculate_mean_rating_a calculate_mean_rating_c(reviews)

Timer unit: 1e-09 s

Total time: 0 s
File: <ipython-input-17-72b63062a01d>
Function: calculate_mean_rating_a at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     5                                           def calculate_mean_rating_a(reviews):
     6                                               count = 0
     7                                               ratings_sum = 0
     8                                               for _, row in reviews.iterrows():
     9                                                   if pd.to_datetime(row['date']).year == 2010:
    10                                                       ratings_sum += row['rating']
    11                                                       count += 1
    12                                               return ratings_sum / count

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [22]:
%%time

def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 12.6 µs


In [23]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-09 s

Total time: 47.0812 s

Could not find file <timed exec>

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1       2849.0   2849.0      0.0  
     3    126679 11585109534.0  91452.5     24.6  
     4    126679 2633481774.0  20788.6      5.6  
     5    126679  663767818.0   5239.8      1.4  
     6   6792010 1664056311.0    245.0      3.5  
     7   6617066 3367575540.0    508.9      7.2  
     8    174944  101751589.0    581.6      0.2  
     9   6792010 4527136763.0    666.5      9.6  
    10                                           
    11         1        976.0    976.0      0.0  
    12    126679 11819918448.0  93306.1     25.1  
    13    126679 1706331597.0  13469.7      3.6  
    14    126679  665208274.0   5251.1      1.4  
    15   6792010 1753453289.0    258.2      3.7  
    16   6792010 6593365517.0    970.8     14.0  
    17         1       1131.0   1131.0      0.0

In [ ]:
%%time

from collections import defaultdict

def get_word_reviews_count(df):
    word_reviews = defaultdict(list)   #defaultdict никогда не вызывает никаких KeyError, поскольку он предоставляет значение по умолчанию для ключа, которого нет в словаре, созданном пользователем
    word_reviews_count = defaultdict(int)
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split()
        unique_words = set(words) 
        for word in unique_words:
            word_reviews[word].append(recipe_id)
            word_reviews_count[word] += 1
        
    return dict(word_reviews_count)


CPU times: user 12 µs, sys: 1 µs, total: 13 µs
Wall time: 17.6 µs


Основные изменения включают в себя:

  * Использование defaultdict из модуля collections, чтобы избежать проверки 
наличия ключа в словаре перед доступом к нему.

  * Вычисление уникальных слов в каждом отзыве с использованием набора, чтобы избежать повторного подсчета одного и того же рецепта.

  * Использование одного цикла для обновления словарей word_reviews и word_reviews_count с сохранением одной итерации по фрейму данных

  * удалилa аргумент в методе split(), поскольку по умолчанию он разбивается на пробелы.

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [25]:
import pandas as pd
import numpy as np
from numba import jit

# удаляем отзывы с нулевым рейтингом.
reviews = reviews[reviews['rating'] > 0]

def mape(ratings):
    """
    Calculate the MAPE for the given ratings.
    """
    mean_rating = np.mean(ratings)
    abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

# Версия 1: Без использования векторизованных операций и методов массивов numpy и без использования numba
def mape_v1(reviews):
    mape_values = []
    for recipe_id in reviews['recipe_id'].unique():
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mape_ = mape(ratings)
        mape_values.append(mape_)
    return mape_values

# Версия 2: Без использования векторизованных операций и методов массивов numpy, но с использованием numba
@jit(nopython=True)
def mape_numba(ratings, mean_rating):
    """
    Calculate the MAPE for the given ratings using numba to speed up the
    calculations.
    """
    n = ratings.shape[0]
    abs_pct_error = np.empty(n)
    for i in range(n):
        abs_pct_error[i] = abs(ratings[i] - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

def mape_v2(reviews):
    mape_values = []
    for recipe_id in reviews['recipe_id'].unique():
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mean_rating = np.mean(ratings)
        mape_ = mape_numba(ratings.values, mean_rating)
        mape_values.append(mape_)
    return mape_values

                                   
# Версия 3: С использованием векторизованных операций и методов массивов numpy, но без использования numba
def mape_v3(reviews):
    recipe_ids = reviews['recipe_id'].unique()
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean')
    mape_values = []
    for i, recipe_id in enumerate(recipe_ids):
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mean_rating = mean_ratings[i]
        abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
        mape = np.mean(abs_pct_error) * 100
        mape_values.append(mape)
    return mape_values

# Версия 4: C использованием векторизованных операций и методов массивов numpy и numba
@jit(nopython=True)
def mape_numba_v2(ratings, mean_rating):
    """
    Calculate the MAPE for the given ratings using vectorization and numba to
    speed up the calculations.
    """
    abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

def mape_v4(reviews):
    recipe_ids = reviews['recipe_id'].unique()
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean').values
    mape_values = np.empty(len(recipe_ids))
    for i, recipe_id in enumerate(recipe_ids):
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating'].values
        mean_rating = mean_ratings[i]
        mape = mape_numba_v2(ratings, mean_rating)
        mape_values[i] = mape
    return mape_values



In [26]:
%lprun -f mape_v1 mape_v1(reviews)


Timer unit: 1e-09 s

Total time: 49.2571 s
File: <ipython-input-25-cc89f8e88c80>
Function: mape_v1 at line 18

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    18                                           def mape_v1(reviews):
    19         1       1751.0   1751.0      0.0      mape_values = []
    20     27440   37917176.0   1381.8      0.1      for recipe_id in reviews['recipe_id'].unique():
    21     27440 23895637741.0 870832.3     48.5          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    22     27440 2098465184.0  76474.7      4.3          ratings = recipe_reviews['rating']
    23     27440 23188017079.0 845044.4     47.1          mape_ = mape(ratings)
    24     27440   37019039.0   1349.1      0.1          mape_values.append(mape_)
    25         1        184.0    184.0      0.0      return mape_values

In [27]:
%lprun -f mape_v2 mape_v2(reviews)

Timer unit: 1e-09 s

Total time: 32.3091 s
File: <ipython-input-25-cc89f8e88c80>
Function: mape_v2 at line 41

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    41                                           def mape_v2(reviews):
    42         1      10435.0  10435.0      0.0      mape_values = []
    43     27440   36888419.0   1344.3      0.1      for recipe_id in reviews['recipe_id'].unique():
    44     27440 23418976148.0 853461.2     72.5          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    45     27440 2089952380.0  76164.4      6.5          ratings = recipe_reviews['rating']
    46     27440 5358250306.0 195271.5     16.6          mean_rating = np.mean(ratings)
    47     27440 1374418784.0  50088.1      4.3          mape_ = mape_numba(ratings.values, mean_rating)
    48     27440   30573424.0   1114.2      0.1          mape_values.append(mape_)
    49         1        192.0    192.0      0.0      return mape_values

In [28]:
%lprun -f mape_v4 mape_v4(reviews)

Timer unit: 1e-09 s

Total time: 27.204 s
File: <ipython-input-25-cc89f8e88c80>
Function: mape_v4 at line 77

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    77                                           def mape_v4(reviews):
    78         1   10388573.0 10388573.0      0.0      recipe_ids = reviews['recipe_id'].unique()
    79         1   37488870.0 37488870.0      0.1      mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean').values
    80         1      11249.0  11249.0      0.0      mape_values = np.empty(len(recipe_ids))
    81     27440   34617592.0   1261.6      0.1      for i, recipe_id in enumerate(recipe_ids):
    82     27440 23952319317.0 872897.9     88.0          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    83     27440 2244780778.0  81806.9      8.3          ratings = recipe_reviews['rating'].values
    84     27440   28721134.0   1046.7      0.1          mean_rating = mean_ratings[i]
    85     27440  852378712.0  31063.4      3.1          mape = mape_numba_v2(ratings, mean_rating)
    86     27440   43280353.0   1577.3      0.2          mape_values[i] = mape
    87         1        267.0    267.0      0.0      return mape_values